In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class SineWaveDataset(Dataset):
    def __init__(self, num_samples, seq_length):
        self.num_samples = num_samples
        self.seq_length = seq_length
        self.data = []
        for _ in range(num_samples):
            phase = np.random.uniform(0, 2 * np.pi)
            self.data.append(np.sin(np.linspace(phase, phase + 2 * np.pi, seq_length)))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32)

# Parameters
num_samples = 1000
seq_length = 50
batch_size = 32

# Create dataset and dataloader
dataset = SineWaveDataset(num_samples, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [3]:
class Generator(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, seq_length):
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(1, x.size(0), self.hidden_dim).to(x.device)
        x = x.unsqueeze(1).repeat(1, self.seq_length, 1)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out.squeeze(2)


In [4]:
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim, seq_length):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim
        self.seq_length = seq_length
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        batch_size = x.size(0)
        
        # Initialize hidden state and cell state
        h0 = torch.zeros(1, batch_size, self.hidden_dim).to(x.device)
        c0 = torch.zeros(1, batch_size, self.hidden_dim).to(x.device)
        
        # LSTM layer
        out, _ = self.lstm(x, (h0, c0))
        
        # We only need the last output for classification
        out = out[:, -1, :]
        
        # Fully connected layer
        out = self.fc(out)
        
        # Sigmoid activation
        out = self.sigmoid(out)
        
        return out

In [5]:
def train_gan(generator, discriminator, dataloader, num_epochs=100):
    criterion = nn.BCELoss()
    optimizer_g = optim.Adam(generator.parameters(), lr=0.001)
    optimizer_d = optim.Adam(discriminator.parameters(), lr=0.001)
    
    for epoch in range(num_epochs):
        for real_data in dataloader:
            batch_size = real_data.size(0)
            real_data = real_data.view(batch_size, seq_length, -1)

            # Train Discriminator
            optimizer_d.zero_grad()
            real_labels = torch.ones(batch_size, 1)
            fake_labels = torch.zeros(batch_size, 1)

            outputs = discriminator(real_data)
            d_loss_real = criterion(outputs, real_labels)
            d_loss_real.backward()

            noise = torch.randn(batch_size, input_dim)
            fake_data = generator(noise)
            outputs = discriminator(fake_data.detach())
            d_loss_fake = criterion(outputs, fake_labels)
            d_loss_fake.backward()
            optimizer_d.step()

            d_loss = d_loss_real + d_loss_fake

            # Train Generator
            optimizer_g.zero_grad()
            noise = torch.randn(batch_size, input_dim)
            fake_data = generator(noise)
            outputs = discriminator(fake_data)
            g_loss = criterion(outputs, real_labels)
            g_loss.backward()
            optimizer_g.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], d_loss: {d_loss.item():.4f}, g_loss: {g_loss.item():.4f}')

input_dim = 10
hidden_dim = 128
output_dim = 1

generator = Generator(input_dim, hidden_dim, output_dim, seq_length)
discriminator = Discriminator(output_dim, hidden_dim, seq_length)

train_gan(generator, discriminator, dataloader, num_epochs=100)


/opt/anaconda3/envs/traffic/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: Skipping device NVIDIA GeForce GT 750M that does not support Metal 2.0 (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1711403210267/work/aten/src/ATen/mps/MPSDevice.mm:101.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


RuntimeError: For unbatched 2-D input, hx and cx should also be 2-D but got (3-D, 3-D) tensors

In [ ]:
dataloader[0]

TypeError: 'DataLoader' object is not subscriptable